In [992]:
import gspread
import pycountry
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from google.auth import default

from __future__ import print_function
from datetime import datetime
from datetime import timedelta
import os.path
import requests
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

import pandas as pd
import numpy as np
import seaborn as sns
import re
import logging
from math import trunc
import json
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import scipy.stats as stats
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

#### 1. Connect to Google Sheets

In [993]:
# Set up scopes and load credentials
scopes = ["https://www.googleapis.com/auth/spreadsheets"]
credentials_path = "wfm-automations.json"
creds = service_account.Credentials.from_service_account_file(credentials_path, scopes=scopes)

# creds.refresh(Request()) #manually refresh the credentials

gc = gspread.authorize(creds)

sheet_id = "1cbW6p8rdMhq9XD5WL-QJNO1UBRQRTcKQBoh6g1QfEv8" # Marketing Forecasts 2024/25 copied file
sheet = gc.open_by_key(sheet_id)
worksheet = sheet.worksheet("actual_budget_2025")

worksheet

<Worksheet 'actual_budget_2025' id:733883154>

### Steps

1. Takie CUM and NET MAUs and transpose it to a sheet

2. Extrapolate at least 2 years of data

3. Map it with the Langauge % split


### Step 1 - Extracting numbers from marketing file

In [994]:
# net maus
cell_range = worksheet.get('A11:O19')
net_mau = pd.DataFrame(cell_range)
net_mau

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Net MAU,Nov 24,Dec 24,Jan 25,Feb 25,Mar 25,Apr 25,May 25,Jun 25,Jul 25,Aug 25,Sep 25,Oct 25,Nov 25,Dec 25
1,Germany,"17,776","20,210","24,733","21,222","24,888","22,424","24,529","24,749","25,586","27,904","33,626","35,281","31,216","26,543"
2,Austria,"1,842","2,451","2,575","2,700","2,962","2,976","3,068","2,864","2,859","2,797","3,363","3,136","2,583","2,233"
3,France,"8,295","7,918","9,308","8,912","11,815","11,610","11,756","10,581","11,781","13,078","15,603","17,330","16,085","11,325"
4,Italy,146,603,-843,-827,-892,-878,-894,"2,141","2,628","4,329","5,637","3,320","2,850","1,639"
5,Spain,"6,643","6,375","8,014","9,009","9,970","10,298","10,932","9,499","10,218","10,506","14,101","16,155","14,648","12,385"
6,Greater Europe,"1,209","1,765","1,070","2,052","2,915","3,735","4,837","5,861","6,760","7,188","10,412","10,583","9,413","5,001"
7,Non-Euro,269,529,574,"1,077",814,487,480,389,356,322,283,361,422,525
8,Total,"36,180","39,852","45,431","44,145","52,472","50,652","54,708","56,084","60,188","66,124","83,025","86,166","77,217","59,651"


In [995]:
#cum maus
cell_range = worksheet.get('A1:O9')
cum_mau = pd.DataFrame(cell_range)
cum_mau

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Cum. Net MAU,Nov 24,Dec 24,Jan 25,Feb 25,Mar 25,Apr 25,May 25,Jun 25,Jul 25,Aug 25,Sep 25,Oct 25,Nov 25,Dec 25
1,Germany,"1,086,666","1,106,876","1,131,609","1,152,831","1,177,719","1,200,143","1,224,672","1,249,421","1,275,007","1,302,911","1,336,537","1,371,818","1,403,034","1,429,577"
2,Austria,"108,248","110,699","113,274","115,974","118,936","121,912","124,980","127,844","130,703","133,500","136,863","139,999","142,582","144,815"
3,France,"446,451","454,369","463,677","472,589","484,404","496,014","507,770","518,351","530,132","543,210","558,813","576,143","592,228","603,553"
4,Italy,"212,951","213,554","212,711","211,884","210,992","210,114","209,220","211,361","213,989","218,318","223,955","227,275","230,125","231,764"
5,Spain,"298,750","305,125","313,139","322,148","332,118","342,416","353,348","362,847","373,065","383,571","397,672","413,827","428,475","440,860"
6,Greater Europe,"209,482","211,248","212,318","214,370","217,285","221,020","225,857","231,718","238,478","245,666","256,078","266,661","276,074","281,075"
7,Non-Euro,"28,821","29,350","29,924","31,001","31,815","32,302","32,782","33,171","33,527","33,849","34,132","34,493","34,915","35,440"
8,Total,"2,391,369","2,431,220","2,476,651","2,520,796","2,573,268","2,623,920","2,678,628","2,734,712","2,794,900","2,861,024","2,944,049","3,030,215","3,107,432","3,167,083"


### Step 2. Transpose data

In [996]:
# create converter sheet to transpose and extrapoalte data
try:
    converter_worksheet = sheet.add_worksheet(title="Converter", rows=100, cols=20)
except gspread.exceptions.APIError:
    converter_worksheet = sheet.worksheet("Converter")
    converter_worksheet.clear()

cum_mau_values = cum_mau.values.tolist()
converter_worksheet.update("A1", cum_mau_values)

/var/folders/ld/t4l42g9d2kndcwfk5qh21smc0000gp/T/ipykernel_9527/1174368924.py:9: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  converter_worksheet.update("A1", cum_mau_values)


{'spreadsheetId': '1cbW6p8rdMhq9XD5WL-QJNO1UBRQRTcKQBoh6g1QfEv8',
 'updatedRange': 'Converter!A1:O9',
 'updatedRows': 9,
 'updatedColumns': 15,
 'updatedCells': 135}

In [997]:
# access the converter sheet data
converter_data = converter_worksheet.get_all_records() 
df_converter = pd.DataFrame(converter_data)
df_converter


,Cum. Net MAU,Nov 24,Dec 24,Jan 25,Feb 25,Mar 25,Apr 25,May 25,Jun 25,Jul 25,Aug 25,Sep 25,Oct 25,Nov 25,Dec 25
0,Germany,1086666,1106876,1131609,1152831,1177719,1200143,1224672,1249421,1275007,1302911,1336537,1371818,1403034,1429577
1,Austria,108248,110699,113274,115974,118936,121912,124980,127844,130703,133500,136863,139999,142582,144815
2,France,446451,454369,463677,472589,484404,496014,507770,518351,530132,543210,558813,576143,592228,603553
3,Italy,212951,213554,212711,211884,210992,210114,209220,211361,213989,218318,223955,227275,230125,231764
4,Spain,298750,305125,313139,322148,332118,342416,353348,362847,373065,383571,397672,413827,428475,440860
5,Greater Europe,209482,211248,212318,214370,217285,221020,225857,231718,238478,245666,256078,266661,276074,281075
6,Non-Euro,28821,29350,29924,31001,31815,32302,32782,33171,33527,33849,34132,34493,34915,35440
7,Total,2391369,2431220,2476651,2520796,2573268,2623920,2678628,2734712,2794900,2861024,2944049,3030215,3107432,3167083


In [998]:
#rename Cum. Net MAU with country
df_converter = df_converter.rename(columns={"Cum. Net MAU": "Country"})

In [999]:
df_converter.columns

Index(['Country', ' Nov 24', ' Dec 24', ' Jan 25', ' Feb 25', ' Mar 25',
       ' Apr 25', ' May 25', ' Jun 25', ' Jul 25', ' Aug 25', ' Sep 25',
       ' Oct 25', ' Nov 25', ' Dec 25'],
      dtype='object')

In [1000]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# First, let's ensure the data is in the correct format to start with
def format_and_forecast_data(df_converter):
    # Create a list of all months (historical + forecast)
    historical_months = df_converter.columns[1:].tolist()  # Skip the Country column
    forecast_months = ['Jan 26', 'Feb 26', 'Mar 26', 'Apr 26', 'May 26', 'Jun 26', 
                      'Jul 26', 'Aug 26', 'Sep 26', 'Oct 26', 'Nov 26', 'Dec 26']
    all_months = historical_months + forecast_months
    
    # Initialize the result DataFrame with the Country column
    result_df = pd.DataFrame({'Country': df_converter['Country']})
    
    # Prepare data for forecasting
    for country in df_converter['Country']:
        # Get historical data for this country
        historical_values = df_converter[df_converter['Country'] == country].iloc[:, 1:].values.flatten()
        
        # Prepare data for linear regression
        X = np.arange(len(historical_values)).reshape(-1, 1)
        y = historical_values
        
        # Fit linear regression model
        model = LinearRegression()
        model.fit(X, y)
        
        # Generate forecasts
        future_X = np.arange(len(historical_values), len(historical_values) + len(forecast_months)).reshape(-1, 1)
        forecasted_values = model.predict(future_X)
        
        # Combine historical and forecasted values
        all_values = np.concatenate([historical_values, forecasted_values])
        
        # Add to result DataFrame
        country_mask = result_df['Country'] == country
        for month, value in zip(all_months, all_values):
            result_df.loc[country_mask, month] = value
    
    return result_df


In [1001]:
# Use the function
final_df = format_and_forecast_data(df_converter)

final_df.iloc[:, 1:] = final_df.iloc[:, 1:].round(1)
final_df

,Country,Nov 24,Dec 24,Jan 25,Feb 25,Mar 25,Apr 25,May 25,Jun 25,Jul 25,...,Mar 26,Apr 26,May 26,Jun 26,Jul 26,Aug 26,Sep 26,Oct 26,Nov 26,Dec 26
0,Germany,1086666.0,1106876.0,1131609.0,1152831.0,1177719.0,1200143.0,1224672.0,1249421.0,1275007.0,...,1497701.9,1524160.6,1550619.3,1577078.0,1603536.7,1629995.4,1656454.1,1682912.8,1709371.5,1735830.2
1,Austria,108248.0,110699.0,113274.0,115974.0,118936.0,121912.0,124980.0,127844.0,130703.0,...,153905.8,156795.7,159685.5,162575.4,165465.3,168355.1,171245.0,174134.9,177024.7,179914.6
2,France,446451.0,454369.0,463677.0,472589.0,484404.0,496014.0,507770.0,518351.0,530132.0,...,634230.0,646497.0,658764.1,671031.1,683298.1,695565.2,707832.2,720099.3,732366.3,744633.3
3,Italy,212951.0,213554.0,212711.0,211884.0,210992.0,210114.0,209220.0,211361.0,213989.0,...,231480.7,233003.3,234526.0,236048.7,237571.4,239094.1,240616.7,242139.4,243662.1,245184.8
4,Spain,298750.0,305125.0,313139.0,322148.0,332118.0,342416.0,353348.0,362847.0,373065.0,...,466305.1,477289.4,488273.7,499258.0,510242.2,521226.5,532210.8,543195.1,554179.4,565163.7
5,Greater Europe,209482.0,211248.0,212318.0,214370.0,217285.0,221020.0,225857.0,231718.0,238478.0,...,291045.1,296814.3,302583.5,308352.6,314121.8,319891.0,325660.2,331429.4,337198.5,342967.7
6,Non-Euro,28821.0,29350.0,29924.0,31001.0,31815.0,32302.0,32782.0,33171.0,33527.0,...,37225.4,37718.8,38212.3,38705.8,39199.3,39692.8,40186.2,40679.7,41173.2,41666.7
7,Total,2391369.0,2431220.0,2476651.0,2520796.0,2573268.0,2623920.0,2678628.0,2734712.0,2794900.0,...,3311892.7,3372277.9,3432663.1,3493048.3,3553433.5,3613818.7,3674203.9,3734589.1,3794974.3,3855359.5


In [1003]:
df_converter.columns

Index(['Country', ' Nov 24', ' Dec 24', ' Jan 25', ' Feb 25', ' Mar 25',
       ' Apr 25', ' May 25', ' Jun 25', ' Jul 25', ' Aug 25', ' Sep 25',
       ' Oct 25', ' Nov 25', ' Dec 25'],
      dtype='object')

### Step 3. Convert it from T&C to language


- The next step is to convert the MAU forecast, which is provided by growth on a monthly basis and per market (not per language), to a forecast which is grouped by language and also per calendar week.

- The tab ‘TnC:Language Mapping’ calculates the mapping of TnC markets into CS Languages, this is done automatically.

"TnC Group", "Month", "Month Start", "MAUs", "% Change"

### Step 3. CS language Mapping

### Step 4. Interpolate the monthly growth rate 

### Step 5. Create the forecast with the columns and format in `weekly remapped MAU FC`